In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import numpy as np
import string
from gensim import corpora
import gensim
from nltk.corpus import words
from gensim.models import LdaModel, LsiModel, Word2Vec, CoherenceModel, TfidfModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import PCA
from collections import defaultdict
from spacy.lang.en import STOP_WORDS
from pprint import pprint
import gensim.sklearn_api.ldamodel as gensimsklearn
from sklearn.manifold import TSNE
from stemming.porter2 import stem
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Topic coherence and its graph, top topics

In [3]:
# configure number of worker for multicore
import multiprocessing
cpu_count = multiprocessing.cpu_count()
worker = cpu_count - 1
print "No of worker to be used = %s"%worker

No of worker to be used = 7


In [4]:
# stop words
stop_wrd = []
with open('stopword.txt') as f:
    for c in f:
        c = c.replace(",","").replace("\n","").replace('"',"").strip()
        stop_wrd.append(c)
stop = list(STOP_WORDS)
print(len(stop))

305


# Creating Functions

In [5]:
# read document using filename
def read_document(file_name):
    print "Reading Document %s"%file_name
    temp = []
    with open(file_name) as f:
        for l in f:
            temp.append(l)
    print "Document Finished Reading"
    temp = clean_document(temp)
    return temp

In [6]:
#pls check this
def clean_document(doc_complete):
    print "Document Cleaning Started"
    doc_clean = []
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    def clean(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        stop_free = " ".join([i for i in punc_free.split() if i not in stop_wrd])
        stop_free = ' '.join(i for i in stop_free.split() if not i.startswith('u0'))
        stop_free = " ".join([i for i in stop_free.lower().split() if not unicode(i, 'utf-8').isnumeric()])
        stemm = ' '.join(stem(i) for i in stop_free.split())
        normalized = " ".join(lemma.lemmatize(word) for word in stemm.split())
        return normalized
    doc_clean = [clean(doc).split() for doc in doc_complete]
    print "Document Cleaning Ended"
    return doc_clean

In [7]:
# return token frequency , clean document (using frequency of token)
def frequency_calculate(doc_clean):
    frequency = defaultdict(int)
    for text in doc_clean:
        for token in text:
            frequency[token] += 1        
    # make new text data by removing out the token that have occurence less then 2
    minimum_freq = 2
    len_of_token = 2
    # doc_clean = [[token for token in text if frequency[token] > minimum_freq]
    #           for text in doc_clean]
    doc_clean = [[token for token in text if len(token) > len_of_token and frequency[token] > minimum_freq]
              for text in doc_clean]
    print "Total Number of distinct keywords in document after cleaning is %s"%len(frequency.keys())
    return frequency, doc_clean

# LDA (Latent Dirichlet Allocation) Model -- Based on Probabilistic Graphical Models 

<div class="alert alert-block alert-info"> <font color='Blue'> Interpreting pyLDAvis output:
<ul> <li>Bubbles on the left-hand side of plot represents a topic. Larger the bubble, the more prevalent is that topic. </li>
<li>Good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant. </li>
<li>Model with too many topics, will have many overlaps, small sized bubbles are clustered in one region of the chart.</li>
<li>The words on the right-hand side are the salient keywords that form the selected topic.</li>
<li>The red bars represent the frequency of a term in a given topic, and the blue bars represent term's frequency across the entire corpus.</li> </ul></font> </div>

In [8]:
def prepare_lda_prerequisite(doc_clean):
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    return dictionary, doc_term_matrix

In [9]:
def find_lda_optimzal_number_topics(dictionary, corpus, texts, limit=101):
    c_v = [0]*limit
    lm_list = []
    for num_topics in range(10, limit):
        lm = gensim.models.ldamulticore.LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v[num_topics] = cm.get_coherence()
        print "Coherence of model with number of topic = %s is %s"%(num_topics,cm.get_coherence())
        
    best_topic = c_v.index(max(c_v))
    print "Optimal Number of Topics in LDA model is %s with cohernce value %s"%(best_topic,max(c_v))
    return best_topic

In [10]:
def run_lda(dictionary,doc_term_matrix,num_topics=100):
    Lda = gensim.models.ldamulticore.LdaMulticore
    ldamodel = Lda(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=20, workers=worker)
    return ldamodel

# <div class="alert alert-block alert-info"><font color='red'> Non-Anonymous User (Neutral Comments) <font> </div> #



In [11]:
# read the document
doc_cleaned = read_document('verified_neutralcomment.txt')

Reading Document verified_neutralcomment.txt
Document Finished Reading
Document Cleaning Started
Document Cleaning Ended


In [12]:
frequency_doc, doc_cleaned = frequency_calculate(doc_cleaned)

Total Number of distinct keywords in document after cleaning is 68745


In [13]:
dictionary, doc_term_matrix = prepare_lda_prerequisite(doc_cleaned)

In [14]:
num_lda_topics =  find_lda_optimzal_number_topics(dictionary,doc_term_matrix,doc_cleaned)

Coherence of model with number of topic = 10 is 0.3884594562097606
Coherence of model with number of topic = 11 is 0.42850638051711853
Coherence of model with number of topic = 12 is 0.41004652238313893
Coherence of model with number of topic = 13 is 0.41985714338506974
Coherence of model with number of topic = 14 is 0.38646275965772
Coherence of model with number of topic = 15 is 0.36067722259668156
Coherence of model with number of topic = 16 is 0.3968195699442264
Coherence of model with number of topic = 17 is 0.3930782587506408
Coherence of model with number of topic = 18 is 0.39643927718244437
Coherence of model with number of topic = 19 is 0.3995560802425593
Coherence of model with number of topic = 20 is 0.40711261779411945
Coherence of model with number of topic = 21 is 0.4084756683368156
Coherence of model with number of topic = 22 is 0.3762579413622374
Coherence of model with number of topic = 23 is 0.4023326442663773
Coherence of model with number of topic = 24 is 0.40166520

In [15]:
ldamodel = run_lda(dictionary,doc_term_matrix,num_lda_topics)

In [16]:
%time
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
56     1.954256        1       1 -0.022411 -0.028711
36     1.888220        1       2 -0.032988 -0.144910
54     1.840893        1       3 -0.099577  0.038847
55     1.778589        1       4  0.089082  0.045234
65     1.731300        1       5 -0.021350 -0.016369
38     1.719302        1       6  0.068783  0.034236
47     1.719008        1       7  0.163546 -0.175174
2      1.716934        1       8  0.031835  0.196318
7      1.679775        1       9  0.115984 -0.084204
68     1.660417        1      10  0.008528 -0.003104
5      1.629133        1      11  0.173570 -0.197301
14     1.628397        1      12  0.063995  0.154068
26     1.624828        1      13 -0.085743 -0.000591
32     1.611729        1      14  0.085889  0.049500
50     1.594162        1      15 -0.100632 -0.038110
40     1.592139        1      16  0.037541  0.022657
16     1.589182        1      17  0.076626  0.046104
9      1.588957        1      18 -0.051687 -0.062951
70     1.579272        1      19  0.003510  0.035669
66     1.568217        1      20  0.079729  0.040001
25     1.562464        1      21 -0.091617 -0.044623
48     1.552338        1      22  0.025019  0.014099
30     1.537789        1      23 -0.066501  0.031974
33     1.525666        1      24  0.050742  0.019035
23     1.523060        1      25  0.016523  0.135735
6      1.507516        1      26 -0.132059 -0.073091
41     1.498336        1      27 -0.075018 -0.023951
57     1.488381        1      28 -0.032087  0.105762
3      1.477203        1      29 -0.086766 -0.040936
17     1.459204        1      30 -0.121772 -0.063397
...         ...      ...     ...       ...       ...
4      1.346316        1      42 -0.049362 -0.011827
18     1.345313        1      43 -0.080999  0.039452
58     1.332994        1      44  0.026470  0.020322
24     1.321267        1      45 -0.093860 -0.040736
63     1.317330        1      46  0.049365  0.031875
35     1.308058        1      47  0.038608  0.069075
27     1.306688        1      48  0.005561 -0.002116
34     1.299265        1      49 -0.008504  0.008930
29     1.291043        1      50 -0.026821  0.000339
39     1.286389        1      51 -0.092057 -0.031160
42     1.271870        1      52  0.058858  0.036163
59     1.265732        1      53 -0.001230 -0.005104
53     1.261302        1      54 -0.054598 -0.029677
52     1.252445        1      55  0.040082  0.031790
44     1.209671        1      56  0.178393 -0.220904
19     1.206180        1      57  0.010669  0.003950
37     1.189053        1      58  0.046054  0.131353
1      1.188141        1      59  0.016137  0.086027
12     1.184644        1      60  0.059713  0.020779
67     1.184559        1      61  0.061277  0.032474
46     1.147553        1      62 -0.013137  0.005166
22     1.115047        1      63  0.055680  0.033462
13     1.102472        1      64 -0.088389 -0.048598
49     1.084303        1      65  0.057607 -0.015317
8      1.054713        1      66 -0.044006 -0.014060
61     1.025689        1      67 -0.054416 -0.009449
69     1.017957        1      68 -0.055194 -0.019631
45     0.975205        1      69 -0.006360  0.007450
28     0.960374        1      70  0.011498  0.007606
43     0.923329        1      71 -0.060369 -0.043835

[71 rows x 5 columns], topic_info=      Category         Freq       Term        Total  loglift  logprob
term                                                                 
5025   Default  9856.000000      india  9856.000000  30.0000  30.0000
9608   Default  6940.000000     govern  6940.000000  29.0000  29.0000
1200   Default  4584.000000       time  4584.000000  28.0000  28.0000
730    Default  3907.000000    countri  3907.000000  27.0000  27.0000
9219   Default  3620.000000      parti  3620.000000  26.0000  26.0000
6081   Default  3231.000000     muslim  3231.000000  25.0000  25.0000
5001   Default  2924.000000       w

In [ ]:
# Topic 1: Border issues (dispute, dialogue)
# Topic 2: Elections (vote, choice, support)
# Topic 3: Crime (issue, charge, proof, solution, prison)
# Topic 4: Politics (respect, flag, compaign)
# Topic 5: Food (noodle, test, eat, complain)
# Topic 6: Economy (finance, debt, trade)
# Topic 7: Corruption (Scam)
# Topic 8: Crime (blame, loot, preach)
# Topic 9: Social (work, target, riot, rally)
# Topic 10: Economy (tax, income, justify)
# Topic 11: Politics (political party, govern, function)
# Topic 12: Politics (scandal, drama, change)
# Topic 13: Socio-political (farmer, purchase, fail, govt.)
# Topic 14: Social (rape, defence, police)
# Topic 15: Monetray (job, money, spend, reserve)
# Topic 16: Border issue (battle, flee, democracy)
# Topic 17: Crime (police, report, incident, night, protest)
# Topic 18: Politics (bribe, worker, health, rape)


# <div class="alert alert-block alert-info"><font color='red'> Anonymous user (Neutral Comments) </font></div>

In [17]:
# read the document
doc_cleaned = read_document('anony_neutralcomment.txt')

Reading Document anony_neutralcomment.txt
Document Finished Reading
Document Cleaning Started
Document Cleaning Ended


In [18]:
frequency_doc, doc_cleaned = frequency_calculate(doc_cleaned)

Total Number of distinct keywords in document after cleaning is 249


In [19]:
dictionary, doc_term_matrix = prepare_lda_prerequisite(doc_cleaned)

In [20]:
num_lda_topics =  find_lda_optimzal_number_topics(dictionary,doc_term_matrix,doc_cleaned)

Coherence of model with number of topic = 10 is 0.540295292393013
Coherence of model with number of topic = 11 is 0.540295292393013
Coherence of model with number of topic = 12 is 0.540295292393013
Coherence of model with number of topic = 13 is 0.5402952923930129
Coherence of model with number of topic = 14 is 0.5402952923930131
Coherence of model with number of topic = 15 is 0.5402952923930131
Coherence of model with number of topic = 16 is 0.5402952923930129
Coherence of model with number of topic = 17 is 0.5402952923930129
Coherence of model with number of topic = 18 is 0.5402952923930129
Coherence of model with number of topic = 19 is 0.540295292393013
Coherence of model with number of topic = 20 is 0.5402952923930131
Coherence of model with number of topic = 21 is 0.540295292393013
Coherence of model with number of topic = 22 is 0.540295292393013
Coherence of model with number of topic = 23 is 0.540295292393013
Coherence of model with number of topic = 24 is 0.5402952923930129
Co

In [21]:
ldamodel = run_lda(dictionary,doc_term_matrix,num_lda_topics)

In [22]:
%time
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      45.450447        1       1 -0.024728 -0.263399
3       9.776540        1       2 -0.170591 -0.123136
9       7.184684        1       3 -0.228023  0.165785
10      6.846846        1       4  0.105435  0.043906
13      5.945946        1       5  0.207146  0.088122
4       4.819821        1       6  0.182627  0.071061
7       4.705440        1       7  0.043956 -0.023581
11      4.594597        1       8 -0.131846  0.191073
8       2.342343        1       9 -0.023761 -0.058120
12      1.666667        1      10  0.007792 -0.018554
6       1.666667        1      11  0.008122 -0.018210
5       1.666667        1      12  0.007940 -0.018301
2       1.666667        1      13  0.007816 -0.018420
1       1.666667        1      14  0.008116 -0.018227, topic_info=     Category      Freq      Term     Total  loglift  logprob
term                                                         
8     Default  3.000000     india  3.000000  19.0000  19.0000
9     Default  5.000000      govt  5.000000  18.0000  18.0000
5     Default  3.000000     point  3.000000  17.0000  17.0000
3     Default  2.000000       cji  2.000000  16.0000  16.0000
14    Default  2.000000   corrupt  2.000000  15.0000  15.0000
16    Default  2.000000      work  2.000000  14.0000  14.0000
2     Default  2.000000     style  2.000000  13.0000  13.0000
0     Default  5.000000      sell  5.000000  12.0000  12.0000
17    Default  5.000000     daili  5.000000  11.0000  11.0000
15    Default  2.000000    happen  2.000000  10.0000  10.0000
1     Default  2.000000     parti  2.000000   9.0000   9.0000
7     Default  2.000000     invit  2.000000   8.0000   8.0000
10    Default  4.000000       buy  4.000000   7.0000   7.0000
11    Default  4.000000      sane  4.000000   6.0000   6.0000
12    Default  4.000000  dealsaid  4.000000   5.0000   5.0000
13    Default  4.000000     stuff  4.000000   4.0000   4.0000
6     Default  4.000000       big  4.000000   3.0000   3.0000
4     Default  4.000000    person  4.000000   2.0000   2.0000
18    Default  4.000000      basi  4.000000   1.0000   1.0000
18     Topic1  3.665576      basi  4.387870   0.6087  -2.2165
10     Topic1  3.665576       buy  4.387870   0.6087  -2.2165
4      Topic1  3.665576    person  4.387870   0.6087  -2.2165
6      Topic1  3.665576       big  4.387870   0.6087  -2.2165
13     Topic1  3.665576     stuff  4.387870   0.6087  -2.2165
12     Topic1  3.665576  dealsaid  4.387870   0.6087  -2.2165
11     Topic1  3.665576      sane  4.387870   0.6087  -2.2165
17     Topic1  3.665576     daili  5.914033   0.3102  -2.2165
0      Topic1  3.665576      sell  5.914060   0.3102  -2.2165
1      Topic1  0.064314     parti  2.778336  -2.9773  -6.2595
15     Topic1  0.064314    happen  2.778368  -2.9773  -6.2595
...       ...       ...       ...       ...      ...      ...
8     Topic13  0.064804     india  3.460270   0.1166  -2.9461
18    Topic13  0.064975      basi  4.387870  -0.1182  -2.9435
13    Topic13  0.064975     stuff  4.387870  -0.1182  -2.9435
12    Topic13  0.064975  dealsaid  4.387870  -0.1182  -2.9435
11    Topic13  0.064975      sane  4.387870  -0.1182  -2.9435
10    Topic13  0.064975       buy  4.387870  -0.1182  -2.9435
6     Topic13  0.064975       big  4.387870  -0.1182  -2.9435
4     Topic13  0.064975    person  4.387870  -0.1182  -2.9435
9     Topic13  0.064975      govt  5.203180  -0.2887  -2.9435
17    Topic13  0.064975     daili  5.914033  -0.4167  -2.9435
0     Topic13  0.064975      sell  5.914060  -0.4167  -2.9435
1     Topic14  0.064873     parti  2.778336   0.3372  -2.9451
15    Topic14  0.064873    happen  2.778368   0.3372  -2.9451
3     Topic14  0.064873       cji  2.783544   0.3353  -2.9451
7     Topic14  0.064873     invit  2.836520   0.3164  -2.9451
14    Topic14  0.065456   corrupt  2.863137   0.3160  -2.9361
2     Topic14  0.064873     style  2.852889   0.3107  -

In [ ]:
# Topic 1: Politics (government, corruption, sell, buy)
# Topic 2: Politics (government, corruption, work, deal, sell, buy)
# Topic 3: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 4: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 5: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 6: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 7: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 8: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 9: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 10: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 11: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 12: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 13: Politics (government, corruption, work, deal, sell, buy, daily)
# Topic 14: Politics (government, corruption, work, deal, sell, buy, daily)


# <div class="alert alert-block alert-info"><font color='red'> Users with Anonymous profiles (Neutral Comments) </font> </div>

In [23]:
# read the document
doc_cleaned = read_document('null_neutralcomment.txt')

Reading Document null_neutralcomment.txt
Document Finished Reading
Document Cleaning Started
Document Cleaning Ended


In [24]:
frequency_doc, doc_cleaned = frequency_calculate(doc_cleaned)

Total Number of distinct keywords in document after cleaning is 4491


In [25]:
dictionary, doc_term_matrix = prepare_lda_prerequisite(doc_cleaned)

In [26]:
num_lda_topics =  find_lda_optimzal_number_topics(dictionary,doc_term_matrix,doc_cleaned)

Coherence of model with number of topic = 10 is 0.4544868092379448
Coherence of model with number of topic = 11 is 0.44635306165959227
Coherence of model with number of topic = 12 is 0.47514247857186814
Coherence of model with number of topic = 13 is 0.4709240796803525
Coherence of model with number of topic = 14 is 0.474509600752434
Coherence of model with number of topic = 15 is 0.46212186218754614
Coherence of model with number of topic = 16 is 0.49127535428535496
Coherence of model with number of topic = 17 is 0.46590443081381255
Coherence of model with number of topic = 18 is 0.4700213278171936
Coherence of model with number of topic = 19 is 0.47062810767834695
Coherence of model with number of topic = 20 is 0.48652990151947045
Coherence of model with number of topic = 21 is 0.4981823166847845
Coherence of model with number of topic = 22 is 0.4853341716269552
Coherence of model with number of topic = 23 is 0.4767173151471924
Coherence of model with number of topic = 24 is 0.480680

In [27]:
ldamodel = run_lda(dictionary,doc_term_matrix,num_lda_topics)

In [28]:
%time
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
24     16.736231        1       1  0.283992  0.086869
0       3.626335        1       2  0.188037  0.024284
20      3.420113        1       3  0.193667  0.041664
57      2.269188        1       4  0.152579 -0.045954
25      1.941754        1       5  0.057453 -0.171027
34      1.884635        1       6  0.088566 -0.023887
63      1.861230        1       7  0.083575 -0.002333
27      1.851588        1       8  0.044171  0.052271
5       1.849798        1       9  0.008213  0.094724
23      1.846375        1      10  0.199657  0.140237
42      1.742461        1      11  0.121630 -0.019725
53      1.693923        1      12  0.034078  0.019613
21      1.642833        1      13  0.022830  0.129649
46      1.587769        1      14  0.007064 -0.103447
9       1.481301        1      15  0.207931  0.129176
39      1.465092        1      16 -0.001694 -0.153309
59      1.458905        1      17 -0.036600  0.069872
7       1.430735        1      18 -0.085117  0.136713
29      1.402687        1      19 -0.061453  0.025947
61      1.400910        1      20 -0.082232 -0.030289
40      1.301074        1      21  0.089345 -0.089889
47      1.299111        1      22 -0.000927  0.058452
52      1.277381        1      23 -0.017097 -0.061407
35      1.271636        1      24  0.038164 -0.033656
43      1.247185        1      25 -0.016464 -0.001228
67      1.218483        1      26  0.024827 -0.116400
14      1.215945        1      27  0.046770 -0.113319
11      1.208816        1      28 -0.021548 -0.036652
13      1.198376        1      29  0.005440 -0.003987
36      1.193337        1      30 -0.052843 -0.094867
...          ...      ...     ...       ...       ...
49      1.047774        1      40 -0.093946  0.047577
56      1.045094        1      41 -0.002746 -0.125838
45      1.033868        1      42  0.026283  0.038810
58      1.027027        1      43 -0.084397  0.101757
68      1.007525        1      44  0.063657  0.058757
17      0.997557        1      45 -0.063467  0.008746
54      0.979402        1      46 -0.056647  0.022520
16      0.976014        1      47 -0.060283  0.083043
44      0.967401        1      48  0.035107  0.048804
15      0.965697        1      49 -0.094784  0.152315
32      0.952887        1      50 -0.042791  0.022634
19      0.933040        1      51 -0.020150 -0.011652
41      0.904843        1      52 -0.057608 -0.016053
1       0.903572        1      53 -0.059490  0.055551
8       0.897825        1      54 -0.104626  0.083613
62      0.773378        1      55 -0.091132  0.147701
6       0.769092        1      56  0.016721 -0.043003
66      0.766319        1      57 -0.053023  0.043326
38      0.763060        1      58 -0.054037 -0.063168
31      0.727959        1      59 -0.081926  0.002945
51      0.706081        1      60 -0.082174  0.008538
18      0.698291        1      61 -0.047601 -0.076303
37      0.693131        1      62 -0.043663 -0.019720
50      0.685747        1      63 -0.064732 -0.114188
3       0.638457        1      64 -0.082656  0.046080
64      0.600487        1      65 -0.092342 -0.053633
2       0.599241        1      66 -0.048634 -0.020011
22      0.597351        1      67 -0.041753  0.002320
60      0.591972        1      68 -0.120816  0.075012
65      0.585293        1      69 -0.068401 -0.018178

[69 rows x 5 columns], topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
827   Default  231.000000      comment  231.000000  30.0000  30.0000
268   Default  388.000000        polic  388.000000  29.0000  29.0000
187   Default  367.000000       regist  367.000000  28.0000  28.0000
498   Default  367.000000      figarez  367.000000  27.0000  27.0000
94    Default  371.000000         hide  371.000000  26.0000  26.0000
390   Default  389.000000          day  389.00000

In [ ]:
# Topic 1: Crime (case, hide, drama, sting operation)
# Topic 2: Technology (Update on whatsapp)
# Topic 3: Entertainment (Song, movie, act, actor)
# Topic 4: Crime (Criminal underground)
# Topic 5: Politics (democracy, election, leader)
# Topic 6: Politics (drama, party)
# Topic 7: Politics (congratulate, people, bill)
# Topic 8: Internal politics and border issues (blame, shame, pay)
# Topic 9: Crime (attack, terror, fool)
# Topic 10: Technology (email, comment, subscribe)
# Topic 11: Crime (court, order, police, rescue)
# Topic 12: Crime (cheat, hit, settle, violence)
# Topic 13: Communal (Ban on muslim organisations)
# Topic 14: Social (racism, slavery, secularism, freedom)
# Topic 15: Social (traitor, land, voter, economy)
# Topic 16: Politics (party, vote, mission)
# Topic 17: Politics (patriot, insult, fight, justice)
# Topic 18: Communal (religion, follow, peace)